# Introduction

In this notebook, we will test one of the simplest coarse-grained potentials, the Lennard-Jones potential (cite LJ paper here). The 12-6 Lennard Jones potential is pairwise, defined as

$U_{LJ}(r) = 4\epsilon \left( \left(\frac{\sigma}{r}\right)^{12} - \left(\frac{\sigma}{r}\right)^{6} \right)$.

Here we will:

- Compare the pairwise energy of two particles at a distance $0.9\sigma$ from each other
- Compare the total potential energy of a simple 2x3x4 simple cubic grid of particles
- Compare the total potential energy of the same simple cubic grid, but for the Weeks-Chandler-Andersen potential (citation here), which is the Lennard-Jones potential but truncated and shifted at $r=2^{1/6}\sigma$.

In [1]:
import itertools
import subprocess
import io
import numpy as np
import signac

In [2]:
def lj_energy(r, epsilon, sigma):
    """Basic, unshifted version of a 12-6 Lennard-Jones potential"""
    result = 4*epsilon*((sigma/float(r))**12 - (sigma/float(r))**6)
    return result

def wca_energy(r, epsilon, sigma):
    """Shifted, truncated Weeks-Chandler-Andersen potential"""
    return (r <= 2.**(1./6)*sigma)*(lj_energy(r, epsilon, sigma) + 1)

# create a 2x3x4 grid of particles of distance 1 from each other
slab_positions = np.array(list(itertools.product(range(2), range(3), range(4))), dtype=np.float32)

project = signac.init_project('lennard_jones')

reference_parameters = dict(package='reference', version=None, rcut=2.5)
print(reference_parameters)
with project.open_job(reference_parameters) as job:
    # the energy is for a pair of particles, so divide the result by 2
    two_particle_energies = 2*[lj_energy(0.9, 1, 1)/2]
    job.document['two_particle_energies'] = two_particle_energies
    
    lj_slab_energy = 0
    for (ri, rj) in itertools.combinations(slab_positions, 2):
        mag_r = np.linalg.norm(rj - ri)
        lj_slab_energy += lj_energy(mag_r, 1, 1)
    job.document['lj_slab_energy'] = lj_slab_energy
    
    wca_slab_energy = 0
    for (ri, rj) in itertools.combinations(slab_positions, 2):
        mag_r = np.linalg.norm(rj - ri)
        wca_slab_energy += wca_energy(mag_r, 1, 1)
    job.document['wca_slab_energy'] = wca_slab_energy
    
    print(job.document)

{'package': 'reference', 'version': None, 'rcut': 2.5}
{'two_particle_energies': [3.3180594766264613, 3.3180594766264613], 'lj_slab_energy': -32.301335619773035, 'wca_slab_energy': 46.0}


# HOOMD-Blue

In [3]:
import hoomd, hoomd.md, hoomd.meta
hoomd.context.initialize('')

HOOMD-blue v2.2.0 DOUBLE HPMC_MIXED SSE SSE2 
Compiled: 11/09/2017
Copyright 2009-2017 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU


In [4]:
box = hoomd.data.boxdim(L=10)
positions = [(0, 0, 0), (0.9, 0, 0)]

with hoomd.context.SimulationContext():
    snapshot = hoomd.data.make_snapshot(len(positions), box)
    snapshot.particles.position[:] = positions
    system = hoomd.init.read_snapshot(snapshot)
    
    nlist = hoomd.md.nlist.cell()
    lj_force = hoomd.md.pair.lj(r_cut=reference_parameters['rcut'], nlist=nlist)
    lj_force.pair_coeff.set('A', 'A', sigma=1, epsilon=1)
    
    # integrate for one step with a timestep size of 0
    hoomd.md.integrate.mode_standard(dt=0)
    nve = hoomd.md.integrate.nve(group=hoomd.group.all())
    hoomd.run(1)
    
    metadata = hoomd.meta.dump_metadata()
    hoomd_parameters = dict(package='hoomd', version=metadata['hoomd'], rcut=reference_parameters['rcut'])
    
    with project.open_job(hoomd_parameters) as job:
        two_particle_energies = [p.net_energy for p in system.particles]
        job.document['two_particle_energies'] = two_particle_energies

notice(2): Group "all" created containing 2 particles


*Warning*: integrate.*: A timestep of less than 0.0 wa

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 2
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
Time 00:00:00 | Step 1 / 1 | TPS 3.15063 | ETA 00:00:00
Average TPS: 3.13928
---------
-- Neighborlist stats:
0 normal updates / 1 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 1 / n_neigh_avg: 0.5
shortest rebuild period: 100
-- Cell list stats:
Dimension: 3, 3, 3
n_min    : 0 / n_max: 2 / n_avg: 0.0740741
** run complete **


s specified


In [5]:
box = hoomd.data.boxdim(L=20)

with hoomd.context.SimulationContext():
    snapshot = hoomd.data.make_snapshot(len(slab_positions), box)
    snapshot.particles.position[:] = slab_positions
    system = hoomd.init.read_snapshot(snapshot)
    
    nlist = hoomd.md.nlist.cell()
    lj_force = hoomd.md.pair.lj(r_cut=reference_parameters['rcut'], nlist=nlist)
    lj_force.pair_coeff.set('A', 'A', sigma=1, epsilon=1)
    
    # integrate for one step with a timestep size of 0
    hoomd.md.integrate.mode_standard(dt=0)
    nve = hoomd.md.integrate.nve(group=hoomd.group.all())
    hoomd.run(1)
    
    metadata = hoomd.meta.dump_metadata()
    hoomd_parameters = dict(package='hoomd', version=metadata['hoomd'], rcut=reference_parameters['rcut'])
    
    with project.open_job(hoomd_parameters) as job:
        slab_particle_energies = [p.net_energy for p in system.particles]
        job.document['lj_slab_energy'] = sum(slab_particle_energies)

notice(2): Group "all" created containing 24 particles


*Warning*: integrate.*: A timestep of less than 0

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 24
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
Time 00:00:00 | Step 1 / 1 | TPS 3.15177 | ETA 00:00:00
Average TPS: 3.13865
---------
-- Neighborlist stats:
0 normal updates / 1 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 17 / n_neigh_avg: 9.66667
shortest rebuild period: 100
-- Cell list stats:
Dimension: 6, 6, 6
n_min    : 0 / n_max: 24 / n_avg: 0.111111
** run complete **


.0 was specified


In [6]:
box = hoomd.data.boxdim(L=20)

with hoomd.context.SimulationContext():
    snapshot = hoomd.data.make_snapshot(len(slab_positions), box)
    snapshot.particles.position[:] = slab_positions
    system = hoomd.init.read_snapshot(snapshot)
    
    nlist = hoomd.md.nlist.cell()
    lj_force = hoomd.md.pair.lj(r_cut=2.**(1./6), nlist=nlist)
    lj_force.pair_coeff.set('A', 'A', sigma=1, epsilon=1)
    lj_force.set_params(mode='shift')
    
    # integrate for one step with a timestep size of 0
    hoomd.md.integrate.mode_standard(dt=0)
    nve = hoomd.md.integrate.nve(group=hoomd.group.all())
    hoomd.run(1)
    
    metadata = hoomd.meta.dump_metadata()
    hoomd_parameters = dict(package='hoomd', version=metadata['hoomd'], rcut=reference_parameters['rcut'])
    
    with project.open_job(hoomd_parameters) as job:
        slab_particle_energies = [p.net_energy for p in system.particles]
        job.document['wca_slab_energy'] = sum(slab_particle_energies)

notice(2): Group "all" created containing 24 particles


*Warning*: integrate.*: A timestep of less than 0.

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 24
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
Time 00:00:00 | Step 1 / 1 | TPS 3.18552 | ETA 00:00:00
Average TPS: 3.17164
---------
-- Neighborlist stats:
0 normal updates / 1 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 9 / n_neigh_avg: 4.33333
shortest rebuild period: 100
-- Cell list stats:
Dimension: 13, 13, 13
n_min    : 0 / n_max: 4 / n_avg: 0.010924
** run complete **


0 was specified


In [7]:
print(hoomd_parameters)

with project.open_job(hoomd_parameters) as job:
    print(job.document)

{'package': 'hoomd', 'version': {'compiler_version': 'gcc 7.2.0', 'cuda_version': [0, 0], 'hoomd_compile_flags': 'DOUBLE HPMC_MIXED SSE SSE2 ', 'hoomd_git_refspec': 'unknown', 'hoomd_git_sha1': '950e9fec2fff2a4eb75b7d211e59104cab10e19a', 'hoomd_version': [2, 2, 0]}, 'rcut': 2.5}
{'two_particle_energies': [3.318061129578318, 3.318061129578318], 'lj_slab_energy': -32.06916703257887, 'wca_slab_energy': 46.0}


# amber, lammps, gromacs, and so on...

Note: MPS has never used lammps a significant amount, so there are probably more intelligent ways to do this.

In [8]:
lammps_executable = 'lmp_mpi'

lammps_version = subprocess.check_output([lammps_executable]).decode().splitlines()[0]
lammps_parameters = dict(package='lammps', version=lammps_version, rcut=reference_parameters['rcut'])

In [9]:
init_text = """
2 atoms

-10 10 xlo xhi
-10 10 ylo yhi
-10 10 zlo zhi

1 atom types

Masses

1 1

Atoms

    1    1    0 0 0
    2    1    0.9 0 0
"""

commands = """
units lj
read_data init.dat
mass * 1.0

pair_style lj/cut 2.5
pair_coeff * * 1 1

compute eng all pe

timestep 0 
run 1
"""

with project.open_job(hoomd_parameters) as job:        
    with open('init.dat', 'w') as init_file:
        init_file.write(init_text)
        
    try:
        lammps = subprocess.Popen([lammps_executable], stdin=subprocess.PIPE, 
                                  stderr=subprocess.PIPE, stdout=subprocess.PIPE)
        (stdout, stderr) = lammps.communicate(commands.encode())
        print(stderr.decode())
        stdout = stdout.decode()
    finally:
        lammps.terminate()

    try:
        stdout_lines = stdout.splitlines()
        thermo_line = [i for (i, line) in enumerate(stdout_lines) if line.startswith('Step')][0]
        thermo_data = dict(zip(stdout_lines[thermo_line].split(), stdout_lines[thermo_line + 1].split()))
    except:
        print(stdout)
    
    with project.open_job(lammps_parameters) as job:
        two_particle_energies = 2*[float(thermo_data['E_pair'])]
        job.document['two_particle_energies'] = two_particle_energies

In [10]:
init_text = """
{} atoms

-10 10 xlo xhi
-10 10 ylo yhi
-10 10 zlo zhi

1 atom types

Masses

1 1

Atoms
""".format(len(slab_positions))

init_text = [init_text]
for i, (x, y, z) in enumerate(slab_positions):
    init_text.append('    {} 1 {} {} {}'.format(i + 1, x, y, z))
init_text = '\n'.join(init_text)

with project.open_job(hoomd_parameters) as job:        
    with open('init.dat', 'w') as init_file:
        init_file.write(init_text)
        
    try:
        lammps = subprocess.Popen([lammps_executable], stdin=subprocess.PIPE, 
                                  stderr=subprocess.PIPE, stdout=subprocess.PIPE)
        (stdout, stderr) = lammps.communicate(commands.encode())
        print(stderr.decode())
        stdout = stdout.decode()
    finally:
        lammps.terminate()

    try:
        stdout_lines = stdout.splitlines()
        thermo_line = [i for (i, line) in enumerate(stdout_lines) if line.startswith('Step')][0]
        thermo_data = dict(zip(stdout_lines[thermo_line].split(), stdout_lines[thermo_line + 1].split()))
    except:
        print(stdout)
    
    with project.open_job(lammps_parameters) as job:
        slab_energy = float(thermo_data['E_pair'])
        job.document['lj_slab_energy'] = slab_energy*len(slab_positions)

In [11]:
init_text = """
{} atoms

-10 10 xlo xhi
-10 10 ylo yhi
-10 10 zlo zhi

1 atom types

Masses

1 1

Atoms
""".format(len(slab_positions))

init_text = [init_text]
for i, (x, y, z) in enumerate(slab_positions):
    init_text.append('    {} 1 {} {} {}'.format(i + 1, x, y, z))
init_text = '\n'.join(init_text)

commands = """
units lj
read_data init.dat
mass * 1.0

pair_style lj/cut 1.122462048309373
pair_modify shift yes
pair_coeff * * 1 1 

compute eng all pe

timestep 0 
run 1
"""

with project.open_job(hoomd_parameters) as job:        
    with open('init.dat', 'w') as init_file:
        init_file.write(init_text)
        
    try:
        lammps = subprocess.Popen([lammps_executable], stdin=subprocess.PIPE, 
                                  stderr=subprocess.PIPE, stdout=subprocess.PIPE)
        (stdout, stderr) = lammps.communicate(commands.encode())
        print(stderr.decode())
        stdout = stdout.decode()
    finally:
        lammps.terminate()

    try:
        stdout_lines = stdout.splitlines()
        thermo_line = [i for (i, line) in enumerate(stdout_lines) if line.startswith('Step')][0]
        thermo_data = dict(zip(stdout_lines[thermo_line].split(), stdout_lines[thermo_line + 1].split()))
    except:
        print(stdout)
    
    with project.open_job(lammps_parameters) as job:
        slab_energy = float(thermo_data['E_pair'])
        job.document['wca_slab_energy'] = slab_energy*len(slab_positions)

In [12]:
print(lammps_parameters)

with project.open_job(lammps_parameters) as job:
    print(job.document)

{'package': 'lammps', 'version': 'LAMMPS (11 Aug 2017)', 'rcut': 2.5}
{'two_particle_energies': [3.3180595, 3.3180595], 'lj_slab_energy': -32.0691672, 'wca_slab_energy': 46.000000799999995}
